## Select by length

此示例选择器根据长度选择要使用的示例。当您担心构建的提示会超过上下文窗口的长度时，这非常有用。对于较长的输入，它将选择较少的示例来包含，而对于较短的输入，它将选择更多的示例。

In [3]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 创建一个反义词的任务示例
examples = [
    {"input": "开心", "output": "伤心"},
    {"input": "高", "output": "矮"},
    {"input": "精力充沛", "output": "没精打采"},
    {"input": "粗", "output": "细"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(
    # 可供选择的示例。
    examples=examples,
    # PromptTemplate 用于格式化示例。
    example_prompt=example_prompt,
    # 格式化示例的最大长度。
    # 长度由下面的 get_text_length 函数测量。
    max_length=25,
    # 用于获取字符串长度的函数，使用
    # 确定要包含哪些示例。被注释掉是因为
    # 如果未指定，则将其作为默认值提供。
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
)
dynamic_prompt = FewShotPromptTemplate(
    # 我们提供了一个ExampleSelector而不是示例。
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入的反义词",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [4]:
#示例输入量较小，因此选择所有示例。
print(dynamic_prompt.format(adjective="big"))

给出每个输入的反义词

Input: 开心
Output: 伤心

Input: 高
Output: 矮

Input: 精力充沛
Output: 没精打采

Input: 粗
Output: 细

Input: big
Output:


In [5]:
# 示例输入较长，因此仅选择一个示例。
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(adjective=long_string))

给出每个输入的反义词

Input: 开心
Output: 伤心

Input: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Output:


In [6]:
# 您也可以将示例添加到示例选择器。
new_example = {"input": "胖", "output": "瘦"}
dynamic_prompt.example_selector.add_example(new_example)
print(dynamic_prompt.format(adjective="热情"))

给出每个输入的反义词

Input: 开心
Output: 伤心

Input: 高
Output: 矮

Input: 精力充沛
Output: 没精打采

Input: 粗
Output: 细

Input: 胖
Output: 瘦

Input: 热情
Output:


In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:11434/v1"
MODEL = "deepseek-r1:1.5b"
chat = ChatOpenAI(
    model= MODEL,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.7,
)

output_parser = StrOutputParser()

chain = dynamic_prompt | chat | output_parser

result=chain.invoke({"adjective":"热情"})
for chunk in result:
    print(chunk, end="", flush=True)  # 实时流式打印

<think>
嗯，我现在得帮用户找出输入单词的反义词。让我仔细看看每个例子，然后想想它们各自的反义词。

第一个是“开心”变“伤心”，这很简单，当然是“悲伤”。第二个“高”变成“矮”，也是明显的，“矮”。第三个，“精力充沛”到“没精打采”，这说明“精力充沛”意味着有丰富的精力，所以反义词应该是没有力量或者不够活跃的状态。第四个“粗”对应“细”，“粗”是指表面的，而“细”是深入的，所以这明显是一个字面意义上的反义词。

接下来第五个，“热烈”变成什么呢？“热烈”通常意味着热情、热闹，那么它的反义词应该是安静或者冷淡的状态。第六个是“慷慨”，这就是指说话或行动的时候语气比较正向，不显得 slackcased，所以它的反义词应该是“慷慨的”就是负面的表达。

不过，我得再仔细检查一下每个例子是否正确理解了反义词的关系。比如，“开心”和“悲伤”确实是相反的意思，没有明显的问题。“高”对应“矮”没错，“精力充沛”到“没精打采”，没问题。接下来的是“粗”和“细”，字面意思完全不符合对方，所以是对的。

然后是“热烈”和“冷淡”，中间可能有温度或热情的感觉，所以对的上。至于“慷慨”，确实是积极和消极的对立面，所以也是正确的。

总结一下，每个输入单词的反义词都正确对应了输出部分，没有问题。我觉得这样应该就解决了用户的需求。
</think>

开心 → 愉忧  
高 → 矮  
精力充沛 → 无精打采  
粗 → 细  
热衷 → 暗示  
热烈 → 严肃

## 最大余弦相似度的嵌入示例
MaxMarginalRelevanceExampleSelector 根据与输入最相似的示例组合来选择示例，同时还针对多样性进行优化。它通过查找与输入具有最大余弦相似度的嵌入示例来实现这一点，然后迭代地添加它们，同时惩罚它们与已选择示例的接近程度。

```
pip install sentence-transformers
pip install faiss-cpu
```

In [10]:
!pip install sentence-transformers

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
! pip install faiss-cpu

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

# embedding在本地


embeddings_path = 'D:\\AInewModels\\BAAI\\bge-m3'
embeddings = HuggingFaceEmbeddings(model_name=embeddings_path)

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# 创建反义词的假装任务的示例。
examples = [
    {"input": "高", "output": "矮"},
    {"input": "精力充沛", "output": "没精打采"},
    {"input": "粗", "output": "细"},
    {"input": "快乐", "output": "悲伤"},
]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_22688\2678929918.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embeddings_path)
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:

# 作用​：从大量示例中选取 k 个最优示例，兼顾：
# ​    相关性​（与输入问题语义相似）
# ​    多样性​（避免选出的示例内容重复）
# ​算法​：基于 ​最大边际相关性（Maximal Marginal Relevance, MMR）​，在相关性和多样性之间取得平衡。
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # 可供选择的示例列表。
    examples,
    # 嵌入类用于生成用于测量语义相似性的嵌入。
    embeddings,
    # VectorStore 类用于存储嵌入并进行相似性搜索。
    FAISS,
    # 要生成的示例数量。
    k=2,
)
mmr_prompt = FewShotPromptTemplate(
    # 提供一个ExampleSelector而不是示例。
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入的反义词",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)
mmr_prompt

FewShotPromptTemplate(input_variables=['adjective'], input_types={}, partial_variables={}, example_selector=MaxMarginalRelevanceExampleSelector(vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001AE009616A0>, k=2, example_keys=None, input_keys=None, vectorstore_kwargs=None, fetch_k=20), example_prompt=PromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, template='Input: {input}\nOutput: {output}'), suffix='Input: {adjective}\nOutput:', prefix='给出每个输入的反义词')

In [14]:
# 输入是一种感觉，所以应该选择快乐/悲伤的例子作为第一个
print(mmr_prompt.format(adjective="担心"))

给出每个输入的反义词

Input: 快乐
Output: 悲伤

Input: 高
Output: 矮

Input: 担心
Output:


In [15]:
chain = mmr_prompt | chat | output_parser

result=chain.invoke({"adjective":"担心"})
for chunk in result:
    print(chunk, end="", flush=True)  # 实时流式打印

<think>
嗯，用户给了三个例子，让我找出每个单词的反义词。第一个是“快乐”对应“悲伤”，这很简单，因为这两个词都是情感上的相反状态。

第二个例子，“高”对应“矮”，这也简单，高就是较大的，矮就是较小的。

第三个例子是“担忧”，输出为空。我得想想，“担忧”是什么意思呢？它通常是指担心 something会不好的时候，比如“担忧下雨了”。那它的反义词应该是相对容易的事情或者事情发生的情况嘛，比如“开心”或“轻松”。

不过用户并没有给出输出，所以我需要填写反义词。最合适的应该是“开心”，因为它和担忧形成情感上的对比，一个是担心，一个是兴奋。

总结一下，每个输入的反义词分别是：
1. 快乐 → 悲伤
2. 高 → 矮
3. 惊心 → 乐观
</think>

输入： 快乐  
输出： 悲伤  

输入： 高  
输出： 矮  

输入： 惊心  
输出： 乐观

## 通过n-gram重叠选择

NGramOverlapExampleSelector 根据 ngram 重叠得分，根据与输入最相似的示例来选择示例并对其进行排序。 ngram 重叠分数是 0.0 到 1.0 之间的浮点数（含 0.0 和 1.0）。

选择器允许设置阈值分数。 ngram 重叠分数小于或等于阈值的示例被排除。默认情况下，阈值设置为 -1.0，因此不会排除任何示例，只会对它们重新排序。将阈值设置为 0.0 将排除与输入没有 ngram 重叠的示例。

In [17]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector.ngram_overlap import NGramOverlapExampleSelector

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# 虚构翻译任务的示例。.
examples = [
    {"input": "See Spot run.", "output": "请参阅现场运行。"},
    {"input": "My dog barks.", "output": "我的狗吠叫。"},
    {"input": "cat can run", "output": "猫会跑"},
]

In [18]:
example_selector = NGramOverlapExampleSelector(
    # 可供选择的示例。
    examples=examples,
    # PromptTemplate 用于格式化示例。
    example_prompt=example_prompt,
    # 选择器停止的阈值。
    # 默认设置为-1.0。
    threshold=-1.0,
    # 对于负阈值：择器按 ngram 重叠分数对示例进行排序，并且不排除任何示例。
    # 对于大于 1.0 的阈值：选择器排除所有示例，并返回一个空列表。
    # 对于阈值等于 0.0:选择器按 ngram 重叠分数对示例进行排序，并排除那些与输入没有 ngram 重叠的内容。
)
dynamic_prompt = FewShotPromptTemplate(
    # 提供了一个ExampleSelector而不是示例。
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="为每个输入提供中文翻译",
    suffix="Input: {sentence}\nOutput:",
    input_variables=["sentence"],
)

In [20]:
# 一个与“cat can run”有大量 ngram 重叠的示例输入。
# 并且与“我的狗吠”没有重叠。
print(dynamic_prompt.format(sentence="cat can run fast."))

为每个输入提供中文翻译

Input: cat can run
Output: 猫会跑

Input: See Spot run.
Output: 请参阅现场运行。

Input: My dog barks.
Output: 我的狗吠叫。

Input: cat can run fast.
Output:


In [21]:
# 您可以设置排除示例的阈值。
# 例如，设置阈值等于0.0
# 排除与输入没有 ngram 重叠的示例。
# 自从“我的狗叫了。” 与“cat can run fast”没有 ngram 重叠。
# 它被排除在外。
example_selector.threshold = 0.0
print(dynamic_prompt.format(sentence="cat can run fast."))

为每个输入提供中文翻译

Input: cat can run
Output: 猫会跑

Input: cat can run fast.
Output:


In [24]:
# 设置小的非零阈值
example_selector.threshold = 0.01
print(dynamic_prompt.format(sentence="cat can play fetch."))

为每个输入提供中文翻译

Input: cat can run
Output: 猫会跑

Input: cat can play fetch.
Output:


In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:11434/v1"
MODEL = "deepseek-r1:1.5b"
chat = ChatOpenAI(
    model= MODEL,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.7,
)

output_parser = StrOutputParser()

chain = dynamic_prompt | chat | output_parser

result=chain.invoke({"sentence":"cat can play fetch."})
for chunk in result:
    print(chunk, end="", flush=True)  # 实时流式打印


<think>
嗯，用户给了两个例子，第一个是英文，第二个是中文。看起来他们希望我用中文来翻译后一个句子。先看看用户给的例子：

输入1：cat can run
输出1：猫会跑

那输入2就是“cat can play fetch.”，需要翻译成中文。首先，“cat”是“猫”，“can”是“能”，“play”是“打”，“fetch”是“打窝币”。所以直接翻译应该是“猫能打窝币。”
</think>

猫能打窝币。